In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os
import importlib
import matplotlib
import glob
import pickle
from tqdm import notebook as tqdm
tqdm.tqdm.pandas()
import seaborn as sns
import pickle
import scikit_posthocs
from typing import Optional
import scipy.stats

In [ ]:
os.makedirs("results_metrics", exist_ok=True)

from matplotlib import rcParams

rcParams["font.family"] = "sans-serif"
rcParams["font.sans-serif"] = ["DejaVu Sans"]

# output text as text and not paths
rcParams["svg.fonttype"] = "none"
rcParams["pdf.fonttype"] = "truetype"

In [ ]:
%cd ../../stimuli_generation
import sg_utils
%cd ../tools/data_analysis

%cd ../..
import tools.mturk.mturk
%cd tools/data_analysis
from utils import utils_data

In [ ]:
def load_data(folder: str, verbose: bool = False):
    files = glob.glob(os.path.join(folder, "*.pkl"))
    scores_dfs = {}
    for file in tqdm.tqdm(files):
        file_name = os.path.basename(file)
        model_name = file_name[len("machine_interpretability_dreamsim_natural_"):][:-4]
        model_name = model_name.replace("timm__hf_hub_timm_", "timm:")
        
        with open(file, "rb") as f:
            data = pickle.load(f)
        df = data["scores"]
        layer_types = data["layer_types"]
        
        inverse_layer_types = {v:k for k,v in layer_types.items()}
        assert len(inverse_layer_types) == len(layer_types)
        
        if len(df) == 0:
            if verbose:
                print(f"Incomplete data for model {model_name}")
            continue

        df["unit"] = pd.to_numeric(df["unit"])
        df["layer_type"] = df["layer_type"].map(lambda x: inverse_layer_types[x])
        scores_dfs[model_name] = df.convert_dtypes()

    return scores_dfs

In [ ]:
def categorize_layer_type(x: str):
    x = x.lower()
    if x in ("conv1d", "linear"):
        return "Linear"
    elif "conv2d" in x:
        return "Conv"
    elif "layernorm" in x:
        return "LayerNorm"
    elif "groupnorm" in x:
        return "GroupNorm"
    elif "batchnorm" in x:
        return "BatchNorm"
    else:
        return x

In [ ]:
def produce_joined_scores(scores_dfs):
    # Ignore first and last layer.
    for k in tqdm.tqdm(scores_dfs):
        forbidden_layers = scores_dfs[k]["layer"].unique()[[0, -1]]
        scores_dfs[k] = scores_dfs[k][~scores_dfs[k]["layer"].isin(forbidden_layers)]

    joined_scores_df = []
    for k in tqdm.tqdm(scores_dfs):
        df = scores_dfs[k].copy()
        df["model"] = k
        df = df.reset_index()
        df = df.set_index(["model", "layer"])    
        joined_scores_df.append(df)
    
    joined_scores_df = pd.concat(joined_scores_df)

    print("Number of models for which data was loaded:", joined_scores_df.index.get_level_values("model").nunique())

    joined_scores_df["layer_type_category"] = joined_scores_df["layer_type"].map(categorize_layer_type)

    return joined_scores_df

In [ ]:
def group_scores(joined_scores_df):
    layer_groups = joined_scores_df.groupby(["model", "layer"])
    joined_layer_means = utils_data.groupby_safe_mean(layer_groups).drop("index", axis=1)
    joined_layer_std = utils_data.groupby_safe_std(layer_groups).drop("index", axis=1)
    
    layer_type_groups = joined_scores_df.groupby(["model", "layer_type"])
    joined_layer_type_means = utils_data.groupby_safe_mean(layer_type_groups).drop("index", axis=1)
    joined_layer_type_std = utils_data.groupby_safe_std(layer_type_groups).drop("index", axis=1)

    return (joined_layer_means, joined_layer_std), (joined_layer_type_means, joined_layer_type_std)

In [ ]:
timm_imagenet_results_df = pd.read_csv("/is/cluster/fast/rzimmermann/interpretability-comparison-data/timm-results-imagenet.csv")
unit_metadata_folder = "/is/cluster/fast/rzimmermann/interpretability-comparison-data/unit-metadata"

def add_model_meta_data(scores_dfs):
    for k in tqdm.tqdm(scores_dfs):
        model_name = k.replace("timm:", "timm__hf_hub_timm_")

        unit_metadata_fn = os.path.join(unit_metadata_folder, f"unit_metadata_{model_name}.pkl")
        with open(unit_metadata_fn, "rb") as f:
            unit_metadata = pickle.load(f)["scores"]

        scores_dfs[k] = scores_dfs[k].set_index(["layer", "unit"])
        unit_metadata = unit_metadata.set_index(["layer", "unit"])
        unit_metadata = unit_metadata[["min_activation", "max_activation", "mean_activation", "median_activation", "std_activation"]]
        unit_metadata = unit_metadata.loc[scores_dfs[k].index]

        scores_dfs[k] = pd.concat((scores_dfs[k], unit_metadata), axis=1)
        scores_dfs[k] = scores_dfs[k].reset_index()

        rows = timm_imagenet_results_df[timm_imagenet_results_df["model"] == k.replace("timm:", "")]
        if len(rows) == 0:
            scores_dfs[k]["imagenet_top1"] = sg_utils.accuracies[k] * 100
            scores_dfs[k]["imagenet_top5"] = np.nan
            scores_dfs[k]["resolution"] = 224
        elif len(rows) == 1:
            meta_row = rows.iloc[0]
            scores_dfs[k]["imagenet_top1"] = meta_row["top1"]
            scores_dfs[k]["imagenet_top5"] = meta_row["top5"]
            scores_dfs[k]["resolution"] = meta_row["img_size"]
        else:
            raise ValueError(k)

        

In [ ]:
scores_dfs = load_data("/is/cluster/fast/rzimmermann/interpretability-comparison-data/machine-interpretability-hard_coded")
add_model_meta_data(scores_dfs)
joined_scores_df = produce_joined_scores(scores_dfs)

scores_hard_dfs = load_data("/is/cluster/fast/rzimmermann/interpretability-comparison-data/machine-interpretability-hard-15k-hard_coded")
add_model_meta_data(scores_hard_dfs)
joined_scores_hard_df = produce_joined_scores(scores_hard_dfs)

In [ ]:
joined_scores_df = produce_joined_scores(scores_dfs)
joined_scores_hard_df = produce_joined_scores(scores_hard_dfs)

In [ ]:
# Mark units that are constant over the ImageNet (train) data set
joined_scores_df["is_constant"] = (joined_scores_df["max_activation"].astype(np.float64) - joined_scores_df["min_activation"].astype(np.float64)).abs() < 1e-8
joined_scores_hard_df["is_constant"] = (joined_scores_hard_df["max_activation"].astype(np.float64) - joined_scores_hard_df["min_activation"].astype(np.float64)).abs() < 1e-8

In [ ]:
print(f"Number of units before removing constant units: {len(joined_scores_df)}")
joined_scores_constant_units_df = joined_scores_df[joined_scores_df["is_constant"]].copy()
joined_scores_df = joined_scores_df[~joined_scores_df["is_constant"]].copy()
print(f"Number of units after removing constant units: {len(joined_scores_df)}")

print(f"[Hard] Number of units before removing constant units: {len(joined_scores_hard_df)}")
joined_scores_hard_constant_units_df = joined_scores_hard_df[joined_scores_hard_df["is_constant"]].copy()
joined_scores_hard_df = joined_scores_hard_df[~joined_scores_hard_df["is_constant"]].copy()
print(f"[Hard] Number of units after removing constant units: {len(joined_scores_hard_df)}")

# Perform Main Analysis and Create Figures

In [ ]:
def analyze_constant_units(joined_scores_df, joined_scores_constant_units_df):
    n_changing_units_df = joined_scores_df.groupby("model").apply(lambda df: len(df))
    n_constant_units_df = joined_scores_constant_units_df.groupby("model").apply(lambda df: len(df))
    n_constant_units_df = n_constant_units_df.align(n_changing_units_df)[0].fillna(0)
    n_units_df =  n_constant_units_df + n_changing_units_df
    ratio_constant_units_df = n_constant_units_df / n_units_df

    ratio_constant_units_df = ratio_constant_units_df.sort_values()

    print(
        "Index of the first model with non-zero ratio:",
        ratio_constant_units_df.reset_index()[ratio_constant_units_df.reset_index()[0] > 0].reset_index().iloc[0]["index"]
    )

    print(
        "Index of the first model with ratio >= 1%:",
        ratio_constant_units_df.reset_index()[ratio_constant_units_df.reset_index()[0] > 0.01].reset_index().iloc[0]["index"]
    )

    print(
        "Index of the first model with ratio >= 5%:",
        ratio_constant_units_df.reset_index()[ratio_constant_units_df.reset_index()[0] > 0.05].reset_index().iloc[0]["index"]
    )
    
    fig, ax = plt.subplots(1, 1, figsize=(4, 3))
    ax.scatter(1 + np.arange(len(ratio_constant_units_df)), ratio_constant_units_df, s=2, clip_on=False, zorder=2)
    ax.spines[["top", "right"]].set_visible(False)
    ax.spines["left"].set_bounds(0, 0.25)
    ax.set_ylim(-0.02, 0.25)
    ax.spines["bottom"].set_bounds(1, len(ratio_constant_units_df) + 1)
    ax.set_xlim(-50, len(ratio_constant_units_df) + 1)
    ax.set_ylabel("Ratio of Constant Units")
    ax.set_xlabel("Model (Sorted by Ratio)")
    ax.grid(axis="y")
    plt.tight_layout()

    display(ratio_constant_units_df)

analyze_constant_units(joined_scores_df, joined_scores_constant_units_df)
plt.savefig("results_metrics/model_comparison_ratio_constant_units.pdf", bbox_inches="tight")
plt.show()

In [ ]:
(joined_layer_means, joined_layer_std), (joined_layer_type_means, joined_layer_type_std) = group_scores(joined_scores_df)
(joined_layer_hard_means, joined_layer_hard_std), (joined_layer_type_hard_means, joined_layer_type_hard_std) = group_scores(joined_scores_hard_df)

In [ ]:
def plot_layer_type_signififance(joined_layer_type_means, ax=None):
    kruskal_result = scipy.stats.kruskal(
        *[joined_layer_type_means.loc[ids, "mis_confidence"].values
          for ids in joined_layer_type_means.groupby('layer_type_category').groups.values()]
    )
    print(f"Result of Krusal-Walllis-Test: {kruskal_result}")
    pc_layer_type_category = scikit_posthocs.posthoc_conover(
        joined_layer_type_means.reset_index(),
        val_col="mis_confidence",
        group_col="layer_type_category",
        p_adjust="holm"
    )
    display(pc_layer_type_category)
        # Format: diagonal, non-significant, p<0.001, p<0.01, p<0.05
    cmap = ['1', '#fb6a4a',  '#08306b',  '#4292c6', '#c6dbef']
    hax,_ = scikit_posthocs.sign_plot(
        pc_layer_type_category, cmap=cmap, linewidth=1, square=True,
        cbar_ax_bbox=[-0.15, -0.15, 0.035, 0.25], ax=ax
    )
    if ax is None:
        plt.show()

In [ ]:
plot_layer_type_signififance(joined_scores_df)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 3))
plot_layer_type_signififance(joined_layer_type_means, ax=ax)
plt.savefig("results_metrics/model_comparison_layer_type_significance_dreamsim_hard_coded_hard.pdf", bbox_inches="tight")

fig, ax = plt.subplots(1, 1, figsize=(3, 3))
plot_layer_type_signififance(joined_layer_type_hard_means, ax=ax)
plt.savefig("results_metrics/model_comparison_layer_type_significance_dreamsim_hard_coded.pdf", bbox_inches="tight")

In [ ]:
def plot_model_ranks_by_layer_type(joined_layer_type_means):
    fig, ax = plt.subplots(1, 1, figsize=(10, 2.4))

    sorted_joined_layer_type_means = joined_layer_type_means.copy().reset_index().set_index(["model", "layer_type_category", "layer_type"])
    sorted_model_names = sorted_joined_layer_type_means.groupby("model").mean().sort_values("mis_confidence").index.values.tolist()
    sorted_joined_layer_type_means["model_index"] = sorted_joined_layer_type_means.index.get_level_values("model").map(lambda x: sorted_model_names.index(x))
    sorted_joined_layer_type_means = sorted_joined_layer_type_means.sort_values("model_index")
    
    layer_types = sorted_joined_layer_type_means.index.get_level_values("layer_type_category")
    for i, lt in enumerate(sorted(layer_types.unique())):
        filtered_sorted_joined_layer_type_means = sorted_joined_layer_type_means[layer_types == lt]
        x = filtered_sorted_joined_layer_type_means["model_index"]
        y = filtered_sorted_joined_layer_type_means["mis_confidence"]
        ax.scatter(x, y, c=f"C{i}", s=2)
    
    ax.set_xlabel("Model (Sorted by Average Score)")
    ax.set_ylabel("Average Per-layer-type MIS")
    ax.set_xlim(-5, len(sorted_model_names) + 5)
    
    ax.spines[["top", "right"]].set_visible(False)
    ax.spines["bottom"].set_bounds(0, len(sorted_model_names))
    handles = [ax.scatter([], [], c=f"C{i}", s=4) for i in range(len(layer_types.unique()))]
    labels = list(sorted(layer_types.unique()))
    ax.legend(handles, labels, frameon=False, handletextpad=0.1, ncol=2)

In [ ]:
plot_model_ranks_by_layer_type(joined_layer_type_means)
plt.tight_layout()
plt.savefig("results_metrics/model_comparison_layer_type.pdf", bbox_inches="tight")    
plt.show()
plot_model_ranks_by_layer_type(joined_layer_type_hard_means)
plt.tight_layout()
plt.savefig("results_metrics/model_comparison_layer_type_hard.pdf", bbox_inches="tight")    
plt.show()

In [ ]:
imi_display_model_names = {
    "clip-vit_b32": "Clip\nViT-B32",
    "convnext_b": "ConvNeXT-B",
    "in1k-vit_b32": "ViT-B32",
    #"resnet50-l2": "",
    "densenet_201": "DenseNet-201",
    "wide_resnet50": "WideResNet-50",
    "resnet50-linf": "Robust\nResNet-50",
    "resnet50": "ResNet-50",
    "clip-resnet50": "Clip\nResNet-50",
    "googlenet": "GoogLeNet"
}

def plot_rank(df, ylabel, use_confidence=True, bars="std", ax=None, color: Optional[str] = None):
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(12, 4))

    key = "mis_confidence" if use_confidence else "mis"
    groups = df[[key]].groupby("model")
    x = np.arange(len(groups))
    group_means = utils_data.groupby_safe_mean(groups)
    y = group_means[key]
    #x_names = group_means.index.values

    y = y.sort_values()
    y_idxs = y.index
    
    
    if bars == "percentile":
        y_5percentile = utils_data.groupby_safe_apply(groups, lambda d: np.percentile(d, 5))[key]
        y_95percentile = utils_data.groupby_safe_apply(groups, lambda d: np.percentile(d, 95))[key]
        y_5percentile = y_5percentile.loc[y_idxs]
        y_95percentile = y_95percentile.loc[y_idxs]
        y_err = np.stack((y - y_5percentile, y_95percentile - y), 0)
    elif bars == "std":
        y_std = utils_data.groupby_safe_std(groups)[key]
        y_std = y_std.loc[y_idxs]
        y_err = np.stack((y_std, y_std), 0)
    else:
        raise ValueError("invalid bars mode.")
    
    #x_names = x_names[np.argsort(y)]
    x_names = np.array(y_idxs)
    y = np.array(y).astype(float)
    
    previous_model_idxs = [idx for idx, v in enumerate(x_names) if not v.startswith("timm")]

    y_err = y_err.astype(float)

    # import pdb; pdb.set_trace()
    ax.fill_between(x, y - y_err[0], y + y_err[1], alpha=0.3, color=color, edgecolor=None)
    # ax.errorbar(x=x, y=y, yerr=y_err, ls="", alpha=0.5, elinewidth=0.75, color=color)
    ax.scatter(x=x, y=y, s=2, color=color, clip_on=False)
    
    for i, idx in enumerate(previous_model_idxs):
        name = x_names[idx]
        if name not in imi_display_model_names:
            print(f"Skipping model {name} as there is no display name available.")
            continue
        name = imi_display_model_names[name]
        ax.text(x[idx], y=y[idx] + (0.0025 if i % 2 == 0 else -0.0025), s=name, ha="center", va="bottom" if i % 2 == 0 else "top", rotation=45)
        ax.scatter(x=x[[idx]], y=y[[idx]], s=5, c="red", marker="^", clip_on=False)
    
    ax.set_xlabel("Model (Sorted by Average Score)")
    ax.set_ylabel(ylabel)
    
    ax.set_xlim(-5, len(x))
    
    ax.spines[["top", "right"]].set_visible(False)
    ax.spines["bottom"].set_bounds(0, len(x))


def plot_rank_bound(df, ylabel, ax=None, use_confidence: bool = True, color: Optional[str] = None, sort_by: str = "average"):
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(12, 4))

    key = "mis_confidence" if use_confidence else "mis"
    groups = df[[key]].groupby("model")
    x = np.arange(len(groups))
    group_means = utils_data.groupby_safe_mean(groups)
    #groups_max = utils_data.groupby_safe_apply(groups, lambda d: d.max())[key]
    #groups_min = utils_data.groupby_safe_apply(groups, lambda d: d.min())[key]
    groups_max = utils_data.groupby_safe_apply(groups, lambda d: np.percentile(d, 95))[key]
    groups_min = utils_data.groupby_safe_apply(groups, lambda d: np.percentile(d, 5))[key]
    if sort_by == "average":
        sort_by = "Average "
        rank = group_means[key]
    elif sort_by == "min":
        sort_by = ""
        rank = groups_min
    elif sort_by == "max":
        sort_by = ""
        rank = groups_max
    else:
        raise ValueError()
    #x_names = group_means.index.values

    rank = rank.sort_values()
    rank_idxs = rank.index

    groups_min = groups_min.loc[rank_idxs]
    groups_max = groups_max.loc[rank_idxs]
    
    x_names = np.array(rank_idxs)
    y_min = np.array(groups_min).astype(float)
    y_max = np.array(groups_max).astype(float)
    
    previous_model_idxs = [idx for idx, v in enumerate(x_names) if not v.startswith("timm")]

    ax.scatter(x=x, y=y_min, s=2, color=color, clip_on=False)
    #ax.scatter(x=x, y=y_max, s=2, color=color)
    
    for i, idx in enumerate(previous_model_idxs):
        name = x_names[idx]
        if name not in imi_display_model_names:
            print(f"Skipping model {name} as there is no display name available.")
            continue
        name = imi_display_model_names[name]
        ax.text(x[idx], y=rank.iloc[idx] + (0.0025 if i % 2 == 0 else -0.0025), s=name, ha="center", va="bottom" if i % 2 == 0 else "top", rotation=45)
        ax.scatter(x=x[[idx]], y=rank.iloc[[idx]], s=5, c="red", marker="^", clip_on=False)

    ax.set_xlabel(f"Model (Sorted by {sort_by}Score)")
    ax.set_ylabel(ylabel)
    
    ax.set_xlim(-5, len(x))
    
    ax.spines[["top", "right"]].set_visible(False)
    ax.spines["bottom"].set_bounds(0, len(x))


def plot_mis_vs_accuracy(df, ylabel, use_confidence=True, ax=None, color: Optional[str] = None, accuracy_key: str = "imagenet_top1"):
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(10, 3))

    key = "mis_confidence" if use_confidence else "mis"
    groups = df[[accuracy_key, key]].groupby("model")
    group_means = utils_data.groupby_safe_mean(groups)
    y = group_means[key]
    x = group_means[accuracy_key]

    x_names = np.array(x.index)    
    
    ax.scatter(x=x, y=y, s=2, color=color, clip_on=False)

    if accuracy_key == "imagenet_top1":
        ax.set_xlabel("ImageNet Top-1 Accuracy")
    elif accuracy_key == "imagenet_top5":
        ax.set_xlabel("ImageNet Top-1 Accuracy")
    else:
        ay.set_xlabel(accuracy_key)
    ax.set_ylabel(ylabel)

    x_min = x.min()
    x_max = x.max()
    ax.set_xlim(x_min - (x_max - x_min) * 0.02, x_max + (x_max - x_min) * 0.0)

    pearson = scipy.stats.pearsonr(x, y)[0]
    spearman = scipy.stats.spearmanr(x, y)[0]

    ax.legend([ax.scatter([], [], c="C0", s=7)], [f"Models\n$\\rho_p = {pearson:.2f}$\n$\\rho_s = {spearman:.2f}$"],
      loc="lower left", bbox_to_anchor=(0, 0.05), edgecolor="white", borderpad=0
    )
    
    #ax.text(62, 0.79, f"$\\rho_p = {pearson:.2f}$\n$\\rho_s = {spearman:.2f}$")
    
    ax.spines[["top", "right"]].set_visible(False)
    ax.set_xticks([60, 65, 70, 75, 80, 85, 90])
    ax.set_yticks([0.78, 0.80, 0.82, 0.84, 0.86, 0.88, 0.90, 0.92])
    ax.spines["left"].set_bounds(0.78, 0.92)
    ax.set_ylim(0.77, 0.92)
    ax.spines["bottom"].set_bounds(60, 90)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 2.5))
plot_mis_vs_accuracy(joined_scores_df, "Average Per-unit MIS", ax=ax)
ax.grid(axis="y")
plt.tight_layout()
plt.savefig("results_metrics/model_comparsion_imagenet_top1_dreamsim_hard_coded.pdf", bbox_inches="tight")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 3), sharex=True)
plot_rank_bound(joined_scores_df, "5th Percentile of Per-unit MIS", ax=ax, color="C0", sort_by="min")

ax.grid(axis="y")
ax.set_ylim(0.42, 0.8)
ax.spines["left"].set_bounds(0.45, 0.8)
plt.tight_layout()
plt.savefig("results_metrics/model_comparsion_5percentile_unit.pdf", bbox_inches="tight")
plt.show()

In [ ]:
bars = "percentile" # "percentile"

fig, ax = plt.subplots(1, 1, figsize=(10, 3), sharex=True)

plot_rank(joined_scores_df, "Average per-unit MIS", bars=bars,ax=ax, color="C0")
#plot_rank(joined_scores_hard_df, "Average per-unit MIS (Hard)", bars=bars,ax=ax, color="C1")

ax.grid(axis="y")
ax.set_ylim(0.6, 1.0)
ax.spines["left"].set_bounds(0.6, 1.0)
plt.tight_layout()
plt.savefig("results_metrics/model_comparsion_average_unit.pdf", bbox_inches="tight")
plt.show()

In [ ]:
bars = "percentile"

fig, axs = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

plot_rank(joined_scores_df, "Average per-unit MIS (Easy)", bars=bars,ax=axs[0], color="C0")
plot_rank(joined_scores_hard_df, "Average per-unit MIS (Hard)", bars=bars, ax=axs[1], color="C1")

axs[0].grid(axis="y")
axs[0].set_ylim(0.6, 1.0)
axs[0].spines["left"].set_bounds(0.6, 1.0)
axs[0].set_xlabel("")

axs[1].grid(axis="y")
axs[1].set_ylim(0.4, 0.8)
axs[1].spines["left"].set_bounds(0.4, 0.8)

plt.tight_layout()
plt.savefig("results_metrics/model_comparsion_average_unit_easy_and_hard.pdf", bbox_inches="tight")
plt.show()

In [ ]:
utils_data.groupby_safe_mean(joined_scores_df.groupby("model"))[["mis_confidence"]].sort_values("mis_confidence")

In [ ]:
# Print all models in LaTeX format.
unique_models = joined_scores_df.index.get_level_values("model").unique().values
print(', '.join([f'\\scriptsize{{{it[len("timm:"):]}}}' for it in unique_models.tolist() if it.startswith("timm:")]).replace("_", "\\_"))

In [ ]:
def plot_highest_variance_distribution_of_rank_plot(df: pd.DataFrame, n: int = 5, use_confidence: bool = True, bars: str = "percentile",
                                                    return_data: bool = False):
    df = df.copy().set_index("unit", append=True)
    key = "mis_confidence" if use_confidence else "mis"
    groups = df[[key]].groupby("model")
    group_means = utils_data.groupby_safe_mean(groups)
    
    if bars == "percentile":
        y_5percentile = utils_data.groupby_safe_apply(groups, lambda d: np.percentile(d, 5))[key]
        y_95percentile = utils_data.groupby_safe_apply(groups, lambda d: np.percentile(d, 95))[key]
        diff = y_95percentile - y_5percentile
    elif bars == "std":
        y_std = utils_data.groupby_safe_std(groups)[key]
        diff = y_std
    else:
        raise ValueError("invalid bars mode.")

    middle_idx = len(diff) // 2
    top_n_idxs = diff.sort_values()[:n].index
    bottom_n_idxs = diff.sort_values()[-n:].index
    middle_n_idxs = diff.sort_values()[middle_idx - int(np.ceil(n / 2)):middle_idx + int(np.ceil(n / 2))].index

    fig, axs = plt.subplots(nrows=3, ncols=n, sharex=True, figsize=(5, 2.4), gridspec_kw = {"wspace": 0.1, "hspace": 0.03})

    for idxs, axs_r in zip((top_n_idxs, middle_n_idxs, bottom_n_idxs), axs):
        for idx, ax in zip(idxs, axs_r):
            ax.hist(df.loc[idx][key], bins=500, density=True)
    
    for ax in axs.flatten():
        ax.spines[["top", "right", "left"]].set_visible(False)
        ax.get_yaxis().set_ticks([])

    axs[-1, n // 2].set_xlabel("Per-unit MIS")
    axs[0, 0].set_ylabel("Lowest")
    axs[1, 0].set_ylabel("Medium")
    axs[2, 0].set_ylabel("Highest")
    fig.text(0.05, 0.37, "Variability", rotation="vertical")

    if return_data:
        return tuple([[df.loc[[idx]][[key]].copy() for idx in idxs] for idxs in (top_n_idxs, middle_n_idxs, bottom_n_idxs)])

highest_variance_distribution_data = plot_highest_variance_distribution_of_rank_plot(joined_scores_df, return_data=True)
#plt.tight_layout()
plt.savefig("results_metrics/model_comparison_per_unit_low_high_variability.pdf", bbox_inches="tight")

In [ ]:
dfs = highest_variance_distribution_data[-1]
for df in dfs:
    df = df.sort_values(by="mis_confidence")
    start_idx = int(len(df) * 0.01)
    display(df.iloc[start_idx:start_idx + 5])

In [ ]:
joined_scores_model_means_df = utils_data.groupby_safe_mean(joined_scores_df.groupby("model"))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(3, 2.5))
ax.scatter(joined_scores_model_means_df["resolution"], joined_scores_model_means_df["mis_confidence"], s=3)
ax.spines[["top", "right"]].set_visible(False)
ax.spines[["bottom"]].set_bounds(100, 500)
ax.set_ylim(.765, .91)
ax.spines["left"].set_bounds(.77, .91)
ax.set_yticks([.77, .79, .81, .83, .85, .87, .89, .91])
ax.set_xlabel("Input Resolution")
ax.set_ylabel("Average MIS")
plt.tight_layout()
plt.savefig("results_metrics/model_comparison_per_model_resolution.pdf", bbox_inches="tight")

In [ ]:
def add_relative_layer_depth_information(
    df: pd.DataFrame,
    model_layer_structure_folder: str = "/is/cluster/fast/rzimmermann/interpretability-comparison-data/model-layer-structure"
):
    structures = {}
    for fn in glob.glob(os.path.join(model_layer_structure_folder, "*")):
        with open(fn, "rb") as f:
            structures[os.path.basename(fn).replace(".pkl", "").replace("__", "_").replace("timm_hf_hub_timm_", "timm:")] = pickle.load(f) 

    depth_map = {k: {n: i for i, n in enumerate(structures[k])} for k in structures}
    relative_depth_map = {k: {n: i / len(structures[k]) for i, n in enumerate(structures[k])} for k in structures}

    def _lookup(map, m: str, l: str):
        if m not in map:
            return -1
        if l not in map[m]:
            return -2
        return map[m][l]
    
    df = df.copy()
 
    df[["layer_depth", "relative_layer_depth"]] = df.progress_apply(
        lambda row: (
            _lookup(depth_map, row.name[0], row.name[1]),
            _lookup(relative_depth_map, row.name[0], row.name[1])
        ), axis=1, result_type='expand')

    df["layer_depth"] = df["layer_depth"].astype(int)

    return df

In [ ]:
joined_layer_means = add_relative_layer_depth_information(joined_layer_means)

In [ ]:
joined_layer_means["layer_width"] = joined_scores_df.groupby(["model", "layer"]).apply(lambda df: len(df))
joined_layer_means["relative_layer_width"] = joined_layer_means["layer_width"] / joined_layer_means[
    ["layer_width"]].groupby("model").max()["layer_width"].align(joined_layer_means["layer_width"])[0]

In [ ]:
key = "relative_layer_width"
bins = 5

_, ax = plt.subplots(1, 1, figsize=(4.5, 2.3))

labels = ("Conv", "Linear", "BatchNorm", ) # "LayerNorm"""GroupNorm"", C3 "C4",
colors = ("C0", "C1", "C2",  )
extended_labels = []
for l, c in zip(labels, colors):
    df = joined_layer_means.copy()
    df["mis_confidence"] = df["mis_confidence"].astype(float)
    
    df = df[df["layer_type_category"] == l]

    # Recompute relative layer width within the selected subgroup
    df["relative_layer_width"] = df["layer_width"] / df[
    ["layer_width"]].groupby("model").max()["layer_width"].align(df["layer_width"])[0]

    if bins == "unique":
        mean_df = utils_data.groupby_safe_mean(df.groupby(key, observed=False))
        std_df = utils_data.groupby_safe_std(df.groupby(key, observed=False))
        percentile5_df = utils_data.groupby_safe_apply(df.groupby(key, observed=False), lambda x: np.percentile(x, 5))
        percentile95_df = utils_data.groupby_safe_apply(df.groupby(key, observed=False), lambda x: np.percentile(x, 95))
    else:
        bin_labels = pd.qcut(df[key], q=bins, labels=np.arange(bins))
        df["bin"] = bin_labels
        mean_df = utils_data.groupby_safe_mean(df.groupby("bin", observed=False))
        percentile5_df = utils_data.groupby_safe_apply(df.groupby("bin", observed=False), lambda x: np.percentile(x, 5))
        percentile95_df = utils_data.groupby_safe_apply(df.groupby("bin", observed=False), lambda x: np.percentile(x, 95))
        std_df = utils_data.groupby_safe_std(df.groupby("bin", observed=False))
        mean_df = mean_df.set_index(key)
        std_df.index = mean_df.index

    extended_labels.append(f"{l}\n$\\rho_s = {scipy.stats.spearmanr(mean_df.index, mean_df['mis_confidence'])[0]:.2f}$"
        f"\n$\\rho_p = {scipy.stats.pearsonr(mean_df.index, mean_df['mis_confidence'])[0]:.2f}$")
    ax.plot(mean_df.index, mean_df["mis_confidence"],
        label=extended_labels[-1], c=c)
    #ax.fill_between(
    #    mean_df.index,
    #    percentile5_df["mis_confidence"],
    #    percentile95_df["mis_confidence"],
    #    alpha=0.1, color=c
    #)
    ax.scatter(mean_df.index, mean_df["mis_confidence"], marker="x", c=c)
ax.set_ylim(0.8175, 0.88)
ax.spines["left"].set_bounds(0.82, 0.88)
ax.spines["bottom"].set_bounds(0, 1)
ax.set_xlim(-0.025, 1.01)
ax.spines[["top", "right"]].set_visible(False)
ax.legend(
    handles=[ax.scatter([], [], color=c, s=20, marker="x") for c in colors],
    labels=extended_labels, labelspacing=1, handletextpad=0, frameon=False, loc="center left", bbox_to_anchor=(1, 0.45)
)
ax.set_xlabel("Relative Layer Width")
ax.set_ylabel("Average Per-unit MIS")
#plt.tight_layout()
plt.savefig("results_metrics/model_comparison_relative_layer_width_per_layer_layer_type_dreamsim_hard_coded.pdf", bbox_inches="tight")

In [ ]:
key = "relative_layer_depth"
bins = 30

_, ax = plt.subplots(1, 1, figsize=(4.5, 3.2))

labels = ("Conv", "Linear", "BatchNorm", "LayerNorm", "GroupNorm")
colors = ("C0", "C1", "C2", "C3", "C4")
extended_labels = []
for l, c in zip(labels, colors):
    df = joined_layer_means.copy()
    df["mis_confidence"] = df["mis_confidence"].astype(float)
    
    df = df[df["layer_type_category"] == l]
    
    bin_labels = pd.qcut(df[key], q=bins, labels=np.arange(bins))
    df["bin"] = bin_labels
    mean_df = utils_data.groupby_safe_mean(df.groupby("bin", observed=False))
    percentile5_df = utils_data.groupby_safe_apply(df.groupby("bin", observed=False), lambda x: np.percentile(x, 5))
    percentile95_df = utils_data.groupby_safe_apply(df.groupby("bin", observed=False), lambda x: np.percentile(x, 95))
    std_df = utils_data.groupby_safe_std(df.groupby("bin", observed=False))
    mean_df = mean_df.set_index(key)
    std_df.index = mean_df.index

    extended_labels.append(f"{l}\n$\\rho_s = {scipy.stats.spearmanr(mean_df.index, mean_df['mis_confidence'])[0]:.2f}$"
        f"\n$\\rho_p = {scipy.stats.pearsonr(mean_df.index, mean_df['mis_confidence'])[0]:.2f}$")
    ax.plot(mean_df.index, mean_df["mis_confidence"],
        label=extended_labels[-1], c=c)
    #ax.fill_between(
    #    mean_df.index,
    #    percentile5_df["mis_confidence"],
    #    percentile95_df["mis_confidence"],
    #    alpha=0.1, color=c
    #)
    ax.scatter(mean_df.index, mean_df["mis_confidence"], marker="x", c=c)
ax.set_ylim(0.755, 0.90)
ax.spines["left"].set_bounds(0.76, 0.90)
ax.spines["bottom"].set_bounds(0, 1)
ax.set_xlim(-0.025, 1)
ax.spines[["top", "right"]].set_visible(False)
ax.legend(
    handles=[ax.scatter([], [], color=c, s=20, marker="x") for c in colors],
    labels=extended_labels, labelspacing=1, handletextpad=0, frameon=False, loc="center left", bbox_to_anchor=(1, 0.45)
)
ax.set_xlabel("Relative Layer Depth")
ax.set_ylabel("Average Per-unit MIS")
#plt.tight_layout()
plt.savefig("results_metrics/model_comparison_relative_layer_depth_per_layer_layer_type_dreamsim_hard_coded.pdf", bbox_inches="tight")

# Compute MIS for units tested in new psychophysics experiments

In [ ]:
# Alternative unit list
# unit_list = {"units": ["mixed5a_1x1_pre_relu_conv__174", "mixed5a_pool_reduce_pre_relu_conv__99", "mixed4e_1x1_pre_relu_conv__138", "mixed4e_1x1_pre_relu_conv__203", "mixed4e_3x3_bottleneck_pre_relu_conv__124", "mixed4e_pool_reduce_pre_relu_conv__9", "mixed5a_1x1_pre_relu_conv__249", "mixed4e_1x1_pre_relu_conv__36", "mixed4e_3x3_pre_relu_conv__56", "mixed4d_3x3_pre_relu_conv__279", "mixed5a_1x1_pre_relu_conv__164", "mixed5a_1x1_pre_relu_conv__91", "mixed4d_3x3_pre_relu_conv__132", "mixed4e_3x3_pre_relu_conv__203", "mixed4d_3x3_pre_relu_conv__232", "mixed4d_3x3_pre_relu_conv__8", "mixed4e_3x3_bottleneck_pre_relu_conv__35", "mixed4d_3x3_pre_relu_conv__114", "mixed4d_1x1_pre_relu_conv__110", "mixed4e_3x3_bottleneck_pre_relu_conv__34", "mixed4d_3x3_bottleneck_pre_relu_conv__45", "mixed4e_pool_reduce_pre_relu_conv__105", "mixed5a_1x1_pre_relu_conv__246", "mixed4e_3x3_pre_relu_conv__93", "mixed4d_3x3_pre_relu_conv__188", "mixed4d_pool_reduce_pre_relu_conv__46", "mixed4c_3x3_pre_relu_conv__26", "mixed4e_3x3_pre_relu_conv__15", "mixed4d_3x3_bottleneck_pre_relu_conv__1", "mixed4c_1x1_pre_relu_conv__65", "mixed4e_3x3_pre_relu_conv__91", "mixed4d_3x3_bottleneck_pre_relu_conv__123", "mixed4e_3x3_pre_relu_conv__171", "mixed4e_3x3_pre_relu_conv__153", "mixed4d_3x3_pre_relu_conv__273", "mixed4c_5x5_pre_relu_conv__61", "mixed5a_3x3_bottleneck_pre_relu_conv__138", "mixed4e_pool_reduce_pre_relu_conv__63", "mixed4b_1x1_pre_relu_conv__121", "mixed4d_3x3_pre_relu_conv__213", "mixed4d_3x3_pre_relu_conv__150", "mixed4b_pool_reduce_pre_relu_conv__33"]}["units"]
unit_list = {"units": ["mixed3a_5x5_pre_relu_conv__19", "mixed3a_3x3_pre_relu_conv__58", "mixed3a_3x3_bottleneck_pre_relu_conv__37", "mixed3b_3x3_bottleneck_pre_relu_conv__42", "mixed3b_5x5_bottleneck_pre_relu_conv__16", "mixed3b_3x3_bottleneck_pre_relu_conv__20", "mixed3a_5x5_pre_relu_conv__24", "mixed3a_1x1_pre_relu_conv__47", "mixed3a_3x3_pre_relu_conv__45", "mixed3b_5x5_pre_relu_conv__38", "mixed3a_pool_reduce_pre_relu_conv__25", "mixed3a_1x1_pre_relu_conv__46", "mixed4a_3x3_pre_relu_conv__122", "mixed4a_3x3_pre_relu_conv__186", "mixed3a_1x1_pre_relu_conv__63", "mixed3b_5x5_bottleneck_pre_relu_conv__9", "mixed3a_1x1_pre_relu_conv__24", "mixed4a_5x5_pre_relu_conv__7", "mixed3b_pool_reduce_pre_relu_conv__61", "mixed5a_5x5_pre_relu_conv__39", "mixed3b_5x5_bottleneck_pre_relu_conv__23", "mixed4a_5x5_pre_relu_conv__40", "mixed4a_3x3_pre_relu_conv__26", "mixed4a_5x5_pre_relu_conv__46", "mixed5b_3x3_pre_relu_conv__187", "mixed3b_5x5_pre_relu_conv__95", "mixed4a_3x3_pre_relu_conv__170", "mixed5b_5x5_pre_relu_conv__94", "mixed4a_3x3_pre_relu_conv__117", "mixed4a_5x5_bottleneck_pre_relu_conv__2", "mixed3a_1x1_pre_relu_conv__57", "mixed5b_pool_reduce_pre_relu_conv__11", "mixed3b_pool_reduce_pre_relu_conv__50", "mixed3a_3x3_pre_relu_conv__70", "mixed4a_5x5_pre_relu_conv__22", "mixed5b_5x5_pre_relu_conv__76", "mixed3a_3x3_pre_relu_conv__65", "mixed3b_1x1_pre_relu_conv__62", "mixed3b_5x5_bottleneck_pre_relu_conv__26", "mixed3a_3x3_bottleneck_pre_relu_conv__85", "mixed4a_3x3_bottleneck_pre_relu_conv__81", "mixed3a_3x3_pre_relu_conv__92"]}["units"]
unit_list = [u.split("__") for u in unit_list]
unit_list = [(u[0], int(u[1])) for u in unit_list]

tmp_df = joined_scores_df.loc["googlenet"]
values = []
for layer, unit in unit_list:
    try:
        tmp2_df = tmp_df.loc[layer]
        tmp2_df = tmp2_df[tmp2_df["unit"] == unit]
        values.append(tmp2_df.iloc[[0]])
    except:
        print("Skipping layer", layer)
tmp_df = pd.concat(values)

print(tmp_df["mis_confidence"].mean())
del tmp_df, tmp2_df

# Plotly

In [ ]:
import plotly.graph_objs as ply

In [ ]:
from matplotlib import colors
def pyplot_color_to_plotly_color(color_name: str, custom_alpha=None) -> str:
    color = colors.to_rgba(color_name)
    if custom_alpha:
        alpha = custom_alpha
    else:
        alpha = color[3]
    return f"rgba({int(color[0] * 255)}, {int(color[1] * 255)}, {int(color[2] * 255)}, {alpha})"

In [ ]:
def analyze_constant_units_plotly(joined_scores_df, joined_scores_constant_units_df):
    n_changing_units_df = joined_scores_df.groupby("model").apply(lambda df: len(df))
    n_constant_units_df = joined_scores_constant_units_df.groupby("model").apply(lambda df: len(df))
    n_constant_units_df = n_constant_units_df.align(n_changing_units_df)[0].fillna(0)
    n_units_df =  n_constant_units_df + n_changing_units_df
    ratio_constant_units_df = n_constant_units_df / n_units_df

    ratio_constant_units_df = ratio_constant_units_df.sort_values()

    print(
        "Index of the first model with non-zero ratio:",
        ratio_constant_units_df.reset_index()[ratio_constant_units_df.reset_index()[0] > 0].reset_index().iloc[0]["index"]
    )

    print(
        "Index of the first model with ratio >= 1%:",
        ratio_constant_units_df.reset_index()[ratio_constant_units_df.reset_index()[0] > 0.01].reset_index().iloc[0]["index"]
    )

    print(
        "Index of the first model with ratio >= 5%:",
        ratio_constant_units_df.reset_index()[ratio_constant_units_df.reset_index()[0] > 0.05].reset_index().iloc[0]["index"]
    )
    
    fig = ply.Figure([
        ply.Scatter(
            x=1 + np.arange(len(ratio_constant_units_df)),
            y=ratio_constant_units_df,
            customdata=ratio_constant_units_df.index.values,
            line=dict(color=pyplot_color_to_plotly_color("C0")),
            mode='markers',
            name="",
        )
    ])
    fig.update_traces(
        hovertemplate="<br>".join([
            "<b>Model</b>: %{customdata}",
            "<b>Ratio</b>: %{y:.4f}"
        ]),
        marker=dict(size=3)
    )
    
    fig.update_layout(
        font_family="DejaVu Sans",
        hoverlabel=dict(
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="white",
            font=dict(color="black")
        )
    )

    fig.update_xaxes(
        title_text="Model (Sorted by Ratio)",
        range=[-5, len(ratio_constant_units_df) + 5],
        showgrid=False,
    )
    fig.update_yaxes(
        title_text="Ratio of Constant Units",
        range=[-0.01, 0.26],
        showgrid=True,
    )

    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
        height=250
    )
    
    fig.show()
    from plotly.offline import plot
    plot(fig, filename="results_metrics/model_comparison_constant_units.html", auto_open=False, config={'responsive': True, 'displaylogo': False},
         include_plotlyjs="cdn", include_mathjax="cdn")
    #print(fig.to_html(full_html=False, include_plotlyjs="cdn", include_mathjax="cdn"))
    

analyze_constant_units_plotly(joined_scores_df, joined_scores_constant_units_df)

In [ ]:
def plot_model_ranks_by_layer_type_plotly(joined_layer_type_means):
    sorted_joined_layer_type_means = joined_layer_type_means.copy().reset_index().set_index(["model", "layer_type_category", "layer_type"])
    sorted_model_names = sorted_joined_layer_type_means.groupby("model").mean().sort_values("mis_confidence").index.values.tolist()
    sorted_joined_layer_type_means["model_index"] = sorted_joined_layer_type_means.index.get_level_values("model").map(lambda x: sorted_model_names.index(x))
    sorted_joined_layer_type_means = sorted_joined_layer_type_means.sort_values("model_index")

    layer_types = sorted_joined_layer_type_means.index.get_level_values("layer_type_category").unique()

    df = joined_layer_type_means.reset_index().pivot_table(columns="layer_type_category", values="mis_confidence", index="model")

    sorted_model_names = joined_layer_type_means.copy().reset_index().set_index(
        ["model", "layer_type_category", "layer_type"]).groupby(
        "model").mean().sort_values("mis_confidence").index.values.tolist()
    df["model_index"] = df.index.get_level_values("model").map(lambda x: sorted_model_names.index(x))
    df = df.sort_values("model_index")

    df = df.fillna(0.0)

    traces = [
        ply.Scatter(
            y=df["model_index"]*0 + 0.8,
            line=dict(color="rgba(0, 0, 0, 0)"),
            #hover_data=df[lt] > 0,
            mode='markers',
            hoverinfo="text",
            text=[f"<b>Model</b>: {v}" for v in df.index.values],
            name="",
            showlegend=False
        )
    ]

    traces += [
        ply.Scatter(
            y=df[lt],
            line=dict(color=pyplot_color_to_plotly_color(f"C{i}", custom_alpha=0.2)),
            mode='markers',
            hoverinfo="skip",
            showlegend=False,
            name=lt,
            marker=dict(size=3),
        )
    for i, lt in enumerate(sorted(layer_types))]

    traces += [
        ply.Scatter(
            y=df[lt],
            line=dict(color=pyplot_color_to_plotly_color(f"C{i}")),
            mode='markers',
            hoverinfo="text",
            text=[f"<b>{lt}</b>: " + (f"{v:.2f}" if v > 0 else "n/A") for v in df[lt]],
            name=lt,
            marker=dict(size=3),
        )
    for i, lt in enumerate(sorted(layer_types))]
    
    fig = ply.Figure(traces)
    
    fig.update_layout(
        hovermode="x unified",
        hoverdistance=1,
        font_family="DejaVu Sans",
        hoverlabel=dict(
            bgcolor="rgba(255,255,255,0.8)"
        )
    )

    fig.update_xaxes(
        title_text="Model (Sorted by Average MIS)",
        range=[-5, len(df) + 5],
        showgrid=False,
    )
    fig.update_yaxes(
        title_text="Average per-layer-type MIS",
        range=[0.745, 0.95],
        showgrid=True,
    )

    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
        height=350,
        legend=dict(
            yanchor="bottom",
            y=0.025,
            xanchor="right",
            x=0.975,
            bgcolor="rgba(255,255,255,0.8)",
            borderwidth=2,
            bordercolor="white"
        )
    )
    
    fig.show()
    from plotly.offline import plot
    plot(fig, filename="results_metrics/model_comparison_layer_types.html", auto_open=False, config={'responsive': True, 'displaylogo': False},
         include_plotlyjs="cdn", include_mathjax="cdn")
    #print(fig.to_html(full_html=False, include_plotlyjs="cdn", include_mathjax="cdn"))
    
plot_model_ranks_by_layer_type_plotly(joined_layer_type_means)

In [ ]:
def plot_mis_vs_accuracy_plotly(df, ylabel, use_confidence=True, color: Optional[str] = "C0", accuracy_key: str = "imagenet_top1"):
    key = "mis_confidence" if use_confidence else "mis"
    groups = df[[accuracy_key, key]].groupby("model")
    group_means = utils_data.groupby_safe_mean(groups)
    y = group_means[key]
    x = group_means[accuracy_key]

    x_names = np.array(x.index)    
    pearson = scipy.stats.pearsonr(x, y)[0]
    spearman = scipy.stats.spearmanr(x, y)[0]

    traces = [
        ply.Scatter(
            x=x,
            y=y,
            line=dict(color=pyplot_color_to_plotly_color(color)),
            mode='markers',
            name="",
            customdata=x_names,
            marker=dict(size=4.5),
            hoverinfo="text",
            text=[f"<b>Model</b>: {mn} <br><b>MIS</b>: {y:.2f} <br><b>Accuracy</b>: {x:.2f}%" for mn,x,y in zip(x_names, x, y)],
        )]
    
    fig = ply.Figure(traces)
    
    fig.update_layout(
        hovermode="x unified",
        hoverdistance=1,
        font_family="DejaVu Sans",
        hoverlabel=dict(
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="white",
        )
    )


    fig.add_annotation(ply.layout.Annotation(
        x=59,
        y=0.94,
        xanchor="left",
        yanchor="top",
        text=f"$\\rho_p = {pearson:.2f} \\\ \\rho_s = {spearman:.2f}$",
        showarrow=False
    ))
    fig.update_xaxes(
        title_text="ImageNet top-1 Accuracy [%]",
        range=[57, 90],
        showgrid=False,
        constrain="domain"
    )
    fig.update_yaxes(
        title_text="Average per-layer-type MIS",
        range=[0.795, 0.95],
        showgrid=True,
        scaleanchor = "x",
        scaleratio = (90-57) / (0.95 - 0.795)
    )
    fig.update_layout(
        #width=450,
        #height=350,
        margin=dict(l=0, r=0, t=0, b=0),
        legend=dict(
            yanchor="bottom",
            y=0.025,
            xanchor="right",
            x=0.975,
            bgcolor="rgba(255,255,255,0.8)",
            borderwidth=2,
            bordercolor="white"
        )
    )
    try:
        fig.show()
    except Exception as ex:
        print("Could not show figure", ex)
    from plotly.offline import plot
    plot(fig, filename="results_metrics/model_comparison_imagenet_accuracy.html", auto_open=False, config={'responsive': True, 'displaylogo': False},
         include_plotlyjs="cdn", include_mathjax="cdn")
    #print(fig.to_html(full_html=False, include_plotlyjs="cdn", include_mathjax="cdn"))

plot_mis_vs_accuracy_plotly(joined_scores_df, "Average Per-unit MIS")

In [ ]:
def plot_rank_plotly(df, ylabel, use_confidence=True, bars="std", color: Optional[str] = "C0"):
    key = "mis_confidence" if use_confidence else "mis"
    groups = df[[key]].groupby("model")
    x = np.arange(len(groups))
    group_means = utils_data.groupby_safe_mean(groups)
    y = group_means[key]
    #x_names = group_means.index.values

    y = y.sort_values()
    y_idxs = y.index
    
    
    if bars == "percentile":
        y_5percentile = utils_data.groupby_safe_apply(groups, lambda d: np.percentile(d, 5))[key]
        y_95percentile = utils_data.groupby_safe_apply(groups, lambda d: np.percentile(d, 95))[key]
        y_5percentile = y_5percentile.loc[y_idxs]
        y_95percentile = y_95percentile.loc[y_idxs]
        y_err = np.stack((y_5percentile, y_95percentile), 0)
    elif bars == "std":
        y_std = utils_data.groupby_safe_std(groups)[key]
        y_std = y_std.loc[y_idxs]
        y_err = np.stack((y - y_std, y + y_std), 0)
    else:
        raise ValueError("invalid bars mode.")
    
    #x_names = x_names[np.argsort(y)]
    x_names = np.array(y_idxs)
    y = np.array(y).astype(float)
    
    previous_model_idxs = [idx for idx, v in enumerate(x_names) if not v.startswith("timm")]

    y_err = y_err.astype(float)

    normal_color = pyplot_color_to_plotly_color(color)
    special_color = "red"
    traces = [
        ply.Scatter(
            x=np.concatenate((x, x[::-1])), # x, then x reversed
            y=np.concatenate((y_err[0], y_err[1][::-1])), # upper, then lower reversed
            fill='toself',
            fillcolor=pyplot_color_to_plotly_color(color, custom_alpha=0.5),
            line=dict(color='rgba(255,255,255,0)'),
            hoverinfo="skip",
            showlegend=False,
            name=""
        ),
        #[normal_color if x_name.startswith("timm:") else special_color for x_name in x_names]
        ply.Scatter(
            x=x[y_idxs.str.startswith("timm:")],
            y=y[y_idxs.str.startswith("timm:")],
            #line=dict(),
            marker=dict(
                size=3,
                color=normal_color
            ),
            mode="markers",
            customdata=x_names[y_idxs.str.startswith("timm:")],
            hoverinfo="text",
            #hovertemplate="<b>Model</b>: %{customdata} <br> <b>MIS</b>: %{y:.4f}",
            text=[f"<b>Model</b>: {mn} <br><b>MIS</b>: {y:.4f}" for mn,y in zip(x_names[y_idxs.str.startswith("timm:")], y[y_idxs.str.startswith("timm:")])],
            showlegend=False,
            name=""
        ),
        ply.Scatter(
            x=x[~y_idxs.str.startswith("timm:")],
            y=y[~y_idxs.str.startswith("timm:")],
            #line=dict(),
            marker=dict(
                size=3,
                color=special_color
            ),
            mode="markers",
            customdata=x_names[~y_idxs.str.startswith("timm:")],
            hoverinfo="text",
            #hovertemplate="<b>Model</b>: %{customdata} <br> <b>MIS</b>: %{y:.4f}",
            text=[f"<b>Model</b>: {mn} <br><b>MIS</b>: {y:.4f}" for mn,y in zip(x_names[~y_idxs.str.startswith("timm:")], y[~y_idxs.str.startswith("timm:")])],
            showlegend=False,
            name=""
        ),
    ]

    fig = ply.Figure(traces)
    
    fig.update_layout(
        hovermode="x unified",
        hoverdistance=1,
        font_family="DejaVu Sans",
        hoverlabel=dict(
            bgcolor="rgba(255,255,255,0.8)",
            bordercolor="white",
            font=dict(color="black")
        ),
    )

    fig.update_xaxes(
        title_text="Model (Sorted by Average MIS)",
        range=[-5, len(x) + 5],
        showgrid=False,
    )
    fig.update_yaxes(
        title_text=ylabel,
        range=[0.695, 1.01] if bars == "std" else [0.6, 1.01],
        showgrid=True,
    )
    fig.update_layout(
        #width=450,
        height=350,
        margin=dict(l=0, r=0, t=0, b=0),
        legend=dict(
            yanchor="bottom",
            y=0.025,
            xanchor="right",
            x=0.975,
            bgcolor="rgba(255,255,255,0.8)",
            borderwidth=2,
            bordercolor="white"
        )
    )
    fig.show()
    from plotly.offline import plot
    plot(fig, filename="results_metrics/model_comparison_average_mis.html", auto_open=False, config={'responsive': True, 'displaylogo': False},
         include_plotlyjs="cdn", include_mathjax="cdn")
    #print(fig.to_html(full_html=False, include_plotlyjs="cdn", include_mathjax="cdn"))
    
plot_rank_plotly(joined_scores_df, "Average per-unit MIS", bars="percentile")